In [ ]:
import pandas as pd
import gensim
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

import matplotlib as mpl
mpl.rc('font',family='Segoe UI Emoji')

from mean_embedding_vectorizer import MeanEmbeddingVectorizer
from tfidf_embedding_vectorizer import TfidfEmbeddingVectorizer

from sklearn import metrics
import itertools

import nltk
nltk.download('twitter_samples')
nltk.download('stopwords')

In [ ]:
import string
import re
 
from nltk.corpus import stopwords 
stopwords_german = stopwords.words('german')
 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('german')
 
from nltk.tokenize import TweetTokenizer
 
# Happy Emoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])
 
# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
# all emoticons (happy + sad)
emoticons = emoticons_happy.union(emoticons_sad)
 
def clean_tweets(tweet):
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
 
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
 
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/[^\s]*', '', tweet)
    
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    
    # replace years with 'ayearzzz'-Token
    tweet = re.sub(r'([1-2][0-9]{3})', r'ayearzzz', tweet)
    
    # replace numbers with 'anumberzzz'-Token, only numbers outside of words
    tweet = re.sub(r'(?<![0-9a-zA-Z])[0-9]+(?![0-9a-zA-Z])', r'anumberzzz', tweet)
 
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True, reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)
 
    tweets_clean = []    
    for word in tweet_tokens:
        if (word not in stopwords_german and # remove stopwords
              word not in emoticons and # remove emoticons
                word not in string.punctuation): # remove punctuation
            #tweets_clean.append(word)
            stem_word = stemmer.stem(word) # stemming word
            tweets_clean.append(stem_word)
    tweets_clean=" ".join(tweets_clean)
    
    # remove numbers that were pulled out of words by tokenizer
    tweets_clean = re.sub(r'(?<![0-9a-zA-Z])[0-9]+(?![0-9a-zA-Z])', r'', tweets_clean)
    
    return tweets_clean
 
custom_tweet = "RT @Twitter @chapagain Hello There! Have a great day. :) #good #morning http://chapagain.com.np"
 
# print cleaned tweet
print (clean_tweets(custom_tweet))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
def get_most_important_features(vectorizer, model, n=5):
    index_to_word = {v:k for k,v in vectorizer.vocabulary_.items()}
    
    # loop for each class
    classes ={}
    for class_index in range(model.coef_.shape[0]):
        word_importances = [(el, index_to_word[i]) for i,el in enumerate(model.coef_[class_index])]
        sorted_coeff = sorted(word_importances, key = lambda x : x[0], reverse=True)
        tops = sorted(sorted_coeff[:n], key = lambda x : x[0])
        bottom = sorted_coeff[-n:]
        classes[class_index] = {
            'tops':tops,
            'bottom':bottom
        }
    return classes

In [ ]:
def plot_important_words_binary(importance, labels, name):
    top_scores = [a[0] for a in importance[0]['tops']]
    top_words = [a[1] for a in importance[0]['tops']]
    
    bottom_scores = [a[0] for a in importance[0]['bottom']]
    bottom_words = [a[1] for a in importance[0]['bottom']]
    
    fig = plt.figure(figsize=(10, 10))  
    y_pos = np.arange(len(top_words))
        
    plt.subplot(121)
    plt.barh(y_pos,bottom_scores, align='center', alpha=0.5)
    plt.title(labels[0], fontsize=20)
    plt.yticks(y_pos, bottom_words, fontsize=14)
    plt.suptitle('Key words', fontsize=16)
    plt.xlabel('Importance', fontsize=20)
    
    plt.subplot(122)
    plt.barh(y_pos,top_scores, align='center', alpha=0.5)
    plt.title(labels[1], fontsize=20)
    plt.yticks(y_pos, top_words, fontsize=14)
    plt.suptitle(name, fontsize=16)
    plt.xlabel('Importance', fontsize=20)
    
    plt.subplots_adjust(wspace=0.8)


In [ ]:
def plot_important_words_multi_class(importance, class_labels, name):
    fig = plt.figure(figsize=(10,10))
    for i in range(len(importance)):
        top_scores = [a[0] for a in importance[i]['tops']]
        top_words = [a[1] for a in importance[i]['tops']]
        
        y_pos = np.arange(len(top_words))
        top_pairs = [(a,b) for a,b in zip(top_words, top_scores)]
        top_pairs = sorted(top_pairs, key=lambda x: x[1])
    
        top_words = [a[0] for a in top_pairs]
        top_scores = [a[1] for a in top_pairs]
        
        subplot = str(int(len(importance)/2)+1)+str(2)+str(i + 1)
        plt.subplot(int(len(importance)/2)+1, 2, i + 1)
        plt.barh(y_pos,top_scores, align='center', alpha=0.5)
        plt.title(class_labels[i], fontsize=20, fontname='Segoe UI Emoji')
        plt.yticks(y_pos, top_words, fontsize=14)
        plt.suptitle(name, fontsize=16)
        plt.xlabel('Importance', fontsize=14)

    plt.subplots_adjust(wspace=0.8, hspace=0.6)


In [ ]:
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.patches as mpatches

def plot_LSA(test_data, test_labels, savepath="PCA_demo.csv", plot=True):
        lsa = TruncatedSVD(n_components=2)
        lsa.fit(test_data)
        lsa_scores = lsa.transform(test_data)
        label_names = set(test_labels)
        color_mapper = {label:idx for idx,label in enumerate(label_names)}
        color_column = [color_mapper[label] for label in test_labels]
        colors = ['orange','blue']
        if plot:
            plt.scatter(lsa_scores[:,0], lsa_scores[:,1], s=8, alpha=.8, c=color_column, cmap=mpl.colors.ListedColormap(colors))
            red_patch = mpatches.Patch(color='orange', label=list(label_names)[0])
            green_patch = mpatches.Patch(color='blue', label=list(label_names)[1])
            plt.legend(handles=[red_patch, green_patch], prop={'size': 30})


In [ ]:
# get corpus token lists
class CorpusSentenceTokenList:
    def __init__(self, corpus):
        self.corpus = corpus

    def __iter__(self):
        for tweet in self.corpus:
            yield tweet.split()

End of definitions

------------------

Start data preparation



In [ ]:
importdf=pd.read_csv('../data/trainingssets/all_emoji_tweets_03_12_18_7_labels_excluded.csv', sep =';', usecols=['tweet_full_text', 'target'])
importdf.dropna(inplace=True)
importdf.reset_index(inplace=True, drop=True)

In [ ]:
all_targets = importdf['target'].astype(str).values.tolist()

In [ ]:
y=[]
for i in range(len(all_targets)):
    #Only use first emoji per tweet for now
    y.append(all_targets[i].split(',')[0])

# for filtering in conversion to binary classification later on
dfy=pd.DataFrame(y)
dfx=pd.DataFrame(importdf['tweet_full_text'])
dfx.columns = range(dfx.shape[1])

In [ ]:
# convert to binary classification
binary_labels=['♥️', '😂']   # two Labels chosen for binary classification

dfy=dfy[dfy.isin(binary_labels)]
dfy.dropna(inplace=True)
dfx=dfx[dfy.isin(binary_labels)]
dfx.dropna(inplace=True)

df=dfx.copy()
df.rename(inplace=True, columns={0: "tweet"})
df['target'] = dfy

# balance classes to 50:50 by dropping appropriate (randomized) fraction of majority class
majority_class='♥️'
class_freq=df['target'].value_counts()
df = df.drop(df[df['target'] == majority_class].sample(frac=(1-class_freq[1]/class_freq[0]), random_state=123).index)

# prepare data for following steps
our_tweets=df['tweet'].astype(str).values.tolist()
y=df['target']
y=np.array(y)

In [ ]:
corpus=[]
for i in range(len(our_tweets)):
    corpus.append(clean_tweets(our_tweets[i]))
corpus

End data preparation

----------------------------

Start Word2Vec preparation

In [ ]:
word2vec_model_path = "../data/word2vec/tweets_until_29_11_18_model3.model"
word2vec_model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_model_path, binary=True)

In [ ]:
#word2vec_model['zwei'] #klass aktion zwei gelbew hang bruck revolution ahu

In [ ]:
#w2vKeyedVectors to dict. Necessary in vectorizer, because w2vKeyedVectors has no attribute 'itervalues'
w2v = {w: vec for w, vec in zip(word2vec_model.index2word, word2vec_model.vectors)} 


In [ ]:
corpus_sentences = CorpusSentenceTokenList(corpus)

In [ ]:
phrases = gensim.models.Phrases(corpus_sentences, threshold=50, min_count=20)

In [ ]:
corpus_phrases = [sentence for sentence in phrases[corpus_sentences]]

End word2vec preparation

--------------------------

Start data visualisation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.4, random_state=0)
pipe_transformer = Pipeline([
    ('vect', MeanEmbeddingVectorizer(w2v))
])
pipe_transformer.fit(X_train, y_train);

In [ ]:
fig = plt.figure(figsize=(16, 16))          
plot_LSA(pipe_transformer.transform(X_train), y_train)
plt.savefig('../figures/tweets_from_03_12_18/binaryclass/word2vec_meanembeddingvectorizer_lsa')
plt.show()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.4, random_state=0)
pipe_transformer = Pipeline([
    ('vect', TfidfEmbeddingVectorizer(w2v))
])
pipe_transformer.fit(X_train, y_train);

In [ ]:
fig = plt.figure(figsize=(16, 16))          
plot_LSA(pipe_transformer.transform(X_train), y_train)
plt.savefig('../figures/tweets_from_03_12_18/binaryclass/word2vec_tfidfembeddingvectorizer_lsa')
plt.show()


For use after arbitrary GridSearch

Needs to be run twice to work? Probably some mistake here


fig = plt.figure(figsize=(16, 16))          
clf = gs_clf.best_estimator_.steps.pop(1)
plot_LSA(gs_clf.best_estimator_.transform(X_train), y_train)
gs_clf.best_estimator_.steps.append(clf)
plt.show()

End data visualisation

--------------------------

Start SGDClassifier

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(corpus_phrases, y, test_size=0.4, random_state=0)
pipe_clf = Pipeline([
    ('vect', TfidfEmbeddingVectorizer(w2v)),
    ('clf', SGDClassifier(random_state=0, 
                          max_iter=100, tol=None))
])

parameters = {
     'clf__loss': ['modified_huber','log'], #modified_huber
     'clf__penalty': ['elasticnet','l2'], #elasticnet
     'clf__alpha': [1e-5],
     'clf__epsilon': [0.01],
     'clf__learning_rate': ['invscaling','optimal'],
     'clf__eta0': [10]
     #'clf__eta0': [1e-4, 0.1],
}
    
gs_clf = GridSearchCV(pipe_clf, parameters, cv=3, iid=False, n_jobs=-1, verbose=1)
gs_clf.fit(X_train, y_train)
predicted = gs_clf.predict(X_test)
np.mean(predicted == y_test)  

Test run: 

Tweets from 29.11:

run1:
    * modified_huber
    * elasticnet
    * alpha=1e-5
    * epsilon=0.01
    * invscaling
    * eta0=10
    * TfidfEmbeddingVectorizer
    * tweets_until_29_11_18_model1.model
    
Score 0.7065608698846522

run2:
    * modified_huber
    * elasticnet
    * alpha=1e-5
    * epsilon=0.01
    * invscaling
    * eta0=10
    * TfidfEmbeddingVectorizer
    * german.model (clean_tweets modified)

Score 0.6587319080155757

run3:
    * modified_huber
    * elasticnet
    * alpha=1e-5
    * epsilon=0.01
    * invscaling
    * eta0=10
    * TfidfEmbeddingVectorizer
    * tweets_until_29_11_18_model3.model

Score 0.7092425244287709

In [ ]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

In [ ]:
plt.figure()
cnf_matrix = confusion_matrix(y_test, predicted)
plot_confusion_matrix(cnf_matrix, classes=binary_labels, normalize=True,
                      title='Confusion matrix, with normalization')
plt.savefig('../figures/tweets_from_03_12_18/binaryclass/word2vec_tfidfembeddingvectorizer_sgdclassifier_confusion_matrix')
plt.show()

End Advanced SGDClassifier

----------------------------

Start Advanced RandomForest

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(corpus, y, test_size=0.4, random_state=0)
pipe_clf = Pipeline([
    ('vect', TfidfEmbeddingVectorizer(w2v)),
    ('clf', RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=0))
])

parameters = {
     #'vect__ngram_range': [(1, 1), (1, 2)],
     #'vect__max_df': [0.9, 1.0],
     #'tfidf__use_idf': (True, False)''
     #'clf__criterion': ['gini', 'entropy'],
     #'clf__max_features': ['log2', 'auto', 0.5],
}
gs_clf = GridSearchCV(pipe_clf, parameters, cv=3, iid=False, n_jobs=-1)
gs_clf.fit(X_train, y_train)
predicted = gs_clf.predict(X_test)
np.mean(predicted == y_test)  

In [ ]:
for param_name in sorted(parameters.keys()):
    print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))

In [ ]:
plt.figure()
cnf_matrix = confusion_matrix(y_test, predicted)
plot_confusion_matrix(cnf_matrix, classes=binary_labels, normalize=True,
                      title='Confusion matrix, with normalization')
plt.savefig('../figures/tweets_from_03_12_18/binaryclass/tfidftransformer_randomforestclassifier_confusion_matrix')
plt.show()

End Advanced Random Forest

----------------------------


End word2vec preparation

-----------------------------

